In [1]:
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
import casadi as cd
from sklearn.linear_model import LinearRegression
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

In [2]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)
    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)
            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
            # Append the dataframe to the list
            df_list.append(df)

bill_path = 'D:\\mlinternship\\IITGuwahatiElectricityBills'
for filename in os.listdir(bill_path):
    # Check if the file has the '.xlsx' extension
    if filename.endswith('.xlsx'):
        file_path = os.path.join(bill_path, filename)
        bill_df = pd.read_excel(file_path)

bill_df['Month'] = pd.to_datetime(bill_df['Month'])
#I assumed its in kilowatts
bill_df['MW'] = bill_df['Number of units of electricity consumed']/1000
bill_df.drop(['Number of units of electricity consumed'], axis=1)
bill_df

,Month,MW
0,2022-01-01,1415.4246
1,2022-02-01,1353.4260
2,2022-03-01,2254.4145
3,2022-04-01,2269.1010
4,2022-05-01,2690.4150
5,2022-06-01,2708.4910
6,2022-07-01,3113.6625
7,2022-08-01,3606.0495
8,2022-09-01,3335.3190
9,2022-10-01,2787.5835


,Month,Number of units of electricity consumed,MW
0,2022-01-01,1415424.6,1415.4246
1,2022-02-01,1353426.0,1353.4260
2,2022-03-01,2254414.5,2254.4145
3,2022-04-01,2269101.0,2269.1010
4,2022-05-01,2690415.0,2690.4150
5,2022-06-01,2708491.0,2708.4910
6,2022-07-01,3113662.5,3113.6625
7,2022-08-01,3606049.5,3606.0495
8,2022-09-01,3335319.0,3335.3190
9,2022-10-01,2787583.5,2787.5835


In [ ]:
temperature_data_path = 'D:\\mlinternship\\iitgdata\\temperaturedata\\report'
temperature_df_list = []
for filename in os.listdir(temperature_data_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(temperature_data_path, filename)
            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
            df = df['DATE(YYYY-MM-DD)']['TIME (UTC)']["TEMP. ('C)"]
            # Append the dataframe to the list
            temperature_df_list.append(df)
temperature_df = pd.concat(temperature_df_list, ignore_index=True)
temperature_df['Time'] = pd.concat([temperature_df['DATE(YYYY-MM-DD)'], temperature_df['TIME (UTC)']], axis=1)
temperature_df=temperature_df.drop(['TIME (UTC)']['DATE(YYYY-MM-DD)'], axis = 1)
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])

In [3]:
#read the power data
power_df = pd.concat(df_list, ignore_index=True)

power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
power_df = power_df[['Time', 'MW']]
power_df['MW'] = power_df['MW'].astype(str)
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df = power_df.sort_values('Time')
power_df.to_csv('power_datacsv.csv')
full_power_df = power_df.copy()

# read the temperature data csv
'''
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df.set_index('Time', inplace=True)
temperature_df['temperature'] = temperature_df['temperature'].interpolate(method='polynomial', order = 5)
temperature_df.reset_index(inplace=True)
'''
# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')
df['temperature'] = df['temperature'].interpolate(method='polynomial', order = 5)
df['Time'] = pd.to_datetime(df['Time'])

full_model_start_time = pd.Timestamp('2023-05-01 00:00:00')
full_model_end_time = pd.Timestamp('2023-06-15 23:00:00')
df = df[(df['Time'] >= full_model_start_time) & (df['Time'] <= full_model_end_time)]
df = df.drop(df[df['MW'] > 20].index)
df = df.sort_values('Time')
df.reset_index(drop=True)
df

,Time,MW,temperature
0,2023-05-01 00:00:00,NaN,27.000000
1,2023-05-01 01:00:00,NaN,41.726953
2,2023-05-01 02:00:00,NaN,37.403189
3,2023-05-01 03:00:00,NaN,30.434306
4,2023-05-01 04:00:00,NaN,27.000000
...,...,...,...
1099,2023-06-15 19:00:00,4.0,25.000000
1100,2023-06-15 20:00:00,4.0,25.000000
1101,2023-06-15 21:00:00,4.0,25.000000
1102,2023-06-15 22:00:00,NaN,25.000000


,Time,MW,temperature
0,2023-05-01 00:00:00,NaN,27.000000
1,2023-05-01 01:00:00,NaN,41.726953
2,2023-05-01 02:00:00,NaN,37.403189
3,2023-05-01 03:00:00,NaN,30.434306
4,2023-05-01 04:00:00,NaN,27.000000
...,...,...,...
1099,2023-06-15 19:00:00,4.0,25.000000
1100,2023-06-15 20:00:00,4.0,25.000000
1101,2023-06-15 21:00:00,4.0,25.000000
1102,2023-06-15 22:00:00,NaN,25.000000


In [4]:
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)
# Concatenate CDH with the original DataFrame using the index
df = pd.concat([df, CDH], axis=1)
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)

numOmegas = 24 * 7
num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)
column_names = ['Time', 'MW', 'temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])
for i in range(0,num_of_rows):
        datetime = df.Time.loc[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x

df.iloc[:,4:]=omegas
DF = df.copy()
df = df.dropna()
df.reset_index(inplace=True, drop = True)
df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2023-05-01 05:00:00,2.41,27.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-05-01 06:00:00,3.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-05-01 07:00:00,3.5,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-05-01 08:00:00,4.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-05-01 09:00:00,4.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2023-06-15 17:00:00,4.2,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,2023-06-15 18:00:00,4.2,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,2023-06-15 19:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
866,2023-06-15 20:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#temperature and behavior part together
#check starting and ending dates of week and adjust accordingly
start_time = pd.Timestamp('2023-05-01 00:00:00')
end_time = pd.Timestamp('2023-06-15 23:00:00')

training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
training_df = df[training_mask]
d = training_df.loc[:,'MW']
phi_temperature = training_df.loc[:,'CDH']
phi_behavior = training_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
temperature_theta = opti.variable()
behavior_theta = opti.variable(168)
total_sum =0
for i in range (0,len(d)):
    phi_behavior_i = (cd.MX(phi_behavior.iloc[i].values))
    phi_behavior_i =  cd.vertcat(phi_behavior_i)
    residual = (d.iloc[i] - ((phi_temperature.iloc[i] * temperature_theta) + cd.dot(phi_behavior_i, behavior_theta)))**2
    total_sum += residual

opti.subject_to(temperature_theta >= 0)
e1 = 3.0
e2 = 3.0

for i in range(0, 167):
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) <= e1)
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) >= -e1)
    opti.subject_to(behavior_theta[i] >= 0)
    opti.subject_to(behavior_theta[i] <= 7)

for i in range(1, 167):
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) <= e2)
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) >= -e2)

#hardcoded for may-june 2023
for i in range(16,18):
    if pd.isna(bill_df.iloc[i]['MW']):
        continue
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_weeks = num_hours / 168.0
    weekly_behavior_energy_use = 0
    temperature_energy_use = 0
    for j in range(len(training_df)):
        temperature_energy_use += temperature_theta * training_df.iloc[j]['CDH']

    for j in range(0, 168):
        weekly_behavior_energy_use += behavior_theta[j]

    weekly_energy_use = weekly_behavior_energy_use +  temperature_energy_use
    opti.subject_to(weekly_energy_use < 1.5 * bill_df.iloc[i]['MW'] / num_weeks)
    opti.subject_to(weekly_energy_use > 0.5 * bill_df.iloc[i]['MW'] / num_weeks)

# Solve the optimization problem
opti.minimize(total_sum)
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()

optimal_temperature_theta = sol.value(temperature_theta)
optimal_behavior_theta = sol.value(behavior_theta)
print("Optimal value of temperature_theta:", optimal_temperature_theta)
print("Optimal values of theta:", optimal_behavior_theta)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2337
Number of nonzeros in Lagrangian Hessian.............:    14365

Total number of variables............................:      169
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [18]:
#final model
training_df = df[training_mask]
cdh = training_df.loc[:, 'CDH']
omegas =  training_df.loc[:,'omega1':'omega168']
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_behavior_theta),axis=0)
phi = training_df.loc[:,'CDH':'omega168']
full_model = LinearRegression(fit_intercept=False)
full_model.fit(phi, d)
full_model.coef_ = c
full_model.intercept_ = 0
yhat = full_model.predict(phi.values)
full_modelScore = full_model.score(phi,d)

temperature_model = LinearRegression(fit_intercept=False)
temperature_model.coef_ = optimal_temperature_theta
temperature_pred = temperature_model.predict(cdh)

behavior_model = LinearRegression(fit_intercept=False)
behavior_model.coef_ = optimal_behavior_theta
behavior_pred = behavior_model.predict(omegas.values)


print ('score for constructed full model on full data: ', full_modelScore)
fig,(ax2) = plt.subplots(nrows=1,figsize=(10,9))
_=ax2.plot(training_df['Time'],behavior_pred,label='behavior dependent', marker = '-')
_=ax2.plot(training_df['Time'],temperature_pred,label='temperature dependent', marker = '-')
_=ax2.plot(training_df['Time'],d,label='meas', marker = 'o')
_=ax2.plot(training_df['Time'],yhat,label='pred:', marker = 'x')
ax2.set_title('measured vs predicted data (full constructed model) (only for non null values)')
_=ax2.legend()
plt.show()

AttributeError: 'Series' object has no attribute 'reshape'

In [7]:
#graph with shaded portion to represent missing values

#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
#actual_data = training_df['MW'].dropna()
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_behavior_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace = True)

fig10,(ax10) = plt.subplots(nrows=1,figsize=(10,9))
first_time = True
j = 0

for i in range(1, len(training_df)):
    if (j < len(actual_df)) and (training_df.iloc[i]['Time'] == actual_df.iloc[j]['Time']):
        j+=1
    else:
        iterator = i
        last_iterator = i-1
        first_time= False
        start =  mdates.date2num(training_df.iloc[last_iterator]['Time'])
        end =  mdates.date2num(training_df.iloc[iterator]['Time'])
        print(f'index: {i} start:{start}, width:{end-start}')
        rect = Rectangle((start, 0), end-start, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)

locator = mdates.AutoDateLocator(minticks=3)
formatter = mdates.AutoDateFormatter(locator)
ax10.xaxis.set_major_locator(locator)
ax10.xaxis.set_major_formatter(formatter)
_=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)
_=ax10.plot(training_df['Time'][training_mask],yhat,label='pred:', marker = 'x', color = 'y', zorder = 5)
ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()


index: 1 start:19478.0, width:0.041666666667879326


index: 2 start:19478.041666666668, width:0.04166666666424135


index: 3 start:19478.083333333332, width:0.041666666667879326


index: 4 start:19478.125, width:0.041666666667879326


index: 25 start:19479.0, width:0.041666666667879326


...

index: 1083 start:19523.083333333332, width:0.041666666667879326


index: 1084 start:19523.125, width:0.041666666667879326


index: 1102 start:19523.875, width:0.041666666667879326


index: 1103 start:19523.916666666668, width:0.04166666666424135


Text(0.5, 1.0, 'measured vs predicted data (full constructed model with NA)')

In [8]:
def custom_date_parser(date_string):
    print(f'{type(date_string)}\t{date_string}')
    if type(date_string) == str:
        return pd.to_datetime(date_string, format='%m-%d-%Y %H:%M:%S')
    else:
        print(date_string.month)
        return date_string

# Specify the path to the main directory containing folders and files
path = "D:\\mlinternship\\iitgdata\\May 01 to May 31 2023\\data33kVsubStation-May-01-2023.xlsx"
#path = "D:\\mlinternship\\iitgdata\\May 01 to May 31 2023\\data33kVsubStation-May-17-2023.xlsx"
# Use the custom date parser function
df1 = pd.read_excel(path, header=3, converters={'Time':custom_date_parser})


<class 'datetime.datetime'>	2023-05-01 00:00:00
5
<class 'datetime.datetime'>	2023-05-01 01:00:00
5
<class 'datetime.datetime'>	2023-05-01 01:59:59.990000
5
<class 'datetime.datetime'>	2023-05-01 02:59:59.985000
5
<class 'datetime.datetime'>	2023-05-01 03:59:59.980000
5
<class 'datetime.datetime'>	2023-05-01 04:59:59.975000
5
<class 'datetime.datetime'>	2023-05-01 05:59:59.970000
5
<class 'datetime.datetime'>	2023-05-01 06:59:59.965000
5
<class 'datetime.datetime'>	2023-05-01 07:59:59.960000
5
<class 'datetime.datetime'>	2023-05-01 08:59:59.955000
5
<class 'datetime.datetime'>	2023-05-01 09:59:59.950000
5
<class 'datetime.datetime'>	2023-05-01 10:59:59.945000
5
<class 'datetime.datetime'>	2023-05-01 11:59:59.940000
5
<class 'datetime.datetime'>	2023-05-01 12:59:59.935000
5
<class 'datetime.datetime'>	2023-05-01 13:59:59.930000
5
<class 'datetime.datetime'>	2023-05-01 14:59:59.925000
5
<class 'datetime.datetime'>	2023-05-01 15:59:59.920000
5
<class 'datetime.datetime'>	2023-05-01 16:59:

In [9]:
df1.dtypes

Time          datetime64[ns]
ESS-9                float64
ESS-1+2              float64
ESS-5                float64
ESS-6                float64
7.5 MVA              float64
kW                   float64
PF                   float64
MW                    object
shift note           float64
kWh                  float64
dtype: object

In [10]:
df1


,Time,ESS-9,ESS-1+2,ESS-5,ESS-6,7.5 MVA,kW,PF,MW,shift note,kWh
0,2023-05-01 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR,NaN,NaN
1,2023-05-01 01:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR,NaN,NaN
2,2023-05-01 01:59:59.990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR,NaN,NaN
3,2023-05-01 02:59:59.985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR,NaN,NaN
4,2023-05-01 03:59:59.980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR,NaN,NaN
5,2023-05-01 04:59:59.975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.41,NaN,NaN
6,2023-05-01 05:59:59.970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
7,2023-05-01 06:59:59.965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
8,2023-05-01 07:59:59.960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
9,2023-05-01 08:59:59.955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
